---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

---

# SageMaker Pipelines: Selective Execution Demo

  In this demo notebook, we will walk a user through,
1/ Setting up a new SageMaker pipeline which consists of Pre-Processing, Training, Evaluation and Model Registration - aka end-2-end ML Pipeline
2/ How to leverage the previously defined end-2-end pipeline for Selective Execution - to selectively run subsets of your pipeline

The flow of this notebook is as follows,
- [Download Dataset](#Download-Dataset)
- [Define a New Pipeline](#Define-a-New-Pipeline)
    - [Setup](#Setup)
    - [PreProcessing](#PreProcessing)
    - [Train](#Train)
    - [Evaluation](#Evaluation)
    - [Create Model](#Create-Model)
    - [Register Model](#Register-Model)
    - [Conditional Check](#Conditional-Check)
- [Execute Pipeline](#Execute-Pipeline)
- [Selective Execution](#Selective-Execution)
    - [Setup](#Setup)
    - [Selective Execution Scenarios](#Selective-Execution-Scenarios)
        - [Execute Single Pipeline Step with Changes to Parameters](#Execute-Single-Pipeline-Step-with-Changes-to-Parameters)
        - [Execute Multiple Contiguous Pipeline Steps or the Entire Pipeline Manually](#Execute-Multiple-Contiguous-Pipeline-Steps-or-the-Entire-Pipeline-Manually)

In [2]:
!python3 -m pip install -U sagemaker==2.162.0


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import boto3
import pprint
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

In [4]:
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
default_bucket_prefix = sagemaker_session.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    s3_dataset_path = f"s3://{default_bucket}/{default_bucket_prefix}/sample-dataset/abalone"
    s3_model_path = f"s3://{default_bucket}/{default_bucket_prefix}/Abalone/models/"
else:
    s3_dataset_path = f"s3://{default_bucket}/sample-dataset/abalone"
    s3_model_path = f"s3://{default_bucket}/Abalone/models/"

sm_client = boto3.client("sagemaker")
s3 = boto3.resource("s3")
model_package_group_name = "Abalone-Models"

# Download Dataset 

In [5]:
!mkdir -p raw-data/

In [6]:
s3.Bucket(f"sagemaker-example-files-prod-{region}").download_file(
    "datasets/tabular/uci_abalone/abalone.csv", "raw-data/abalone.csv"
)

In [7]:
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path="raw-data/abalone.csv",
    desired_s3_uri=s3_dataset_path,
)
print(input_data_uri)

s3://sagemaker-us-east-1-681580474417/sample-dataset/abalone/abalone.csv


# Define a New Pipeline

## Setup

In [8]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
from sagemaker.workflow.steps import CacheConfig

Lets define parameters which would be leveraged throught our pipeline, we can change these parameters during execution and control various aspects of our pipeline

In [9]:
step_cache_config = CacheConfig(
    enable_caching=True,
    expire_after="T12H",
)
# processing params
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.c5.xlarge"
)
# training params
train_instance_count = ParameterInteger(name="TrainingInstanceCount", default_value=1)
train_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.c5.xlarge")

xgb_num_round = ParameterInteger(name="XGBNumRounds", default_value=50)
xgb_max_depth = ParameterInteger(name="XGBMaxDepth", default_value=5)
xgb_eta = ParameterFloat(name="XGBETA", default_value=0.2)
xgb_gamma = ParameterInteger(name="XGBGamma", default_value=4)
xgb_min_child_weight = ParameterInteger(name="XGBMinChildWeight", default_value=6)
xgb_subsample = ParameterFloat(name="XGBSubSample", default_value=0.7)

# model params
model_path = ParameterString(
    name="ModelPath", default_value=s3_model_path"
)

default_model_status = ParameterString(name="ApprovalStatus", default_value="PendingManualApproval")

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

## PreProcessing 

In [10]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

In [11]:
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="abalone-preprocess",
    role=role,
    sagemaker_session=pipeline_session,
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


In [12]:
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/output/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/output/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/output/test"),
    ],
    code="code/preprocessing.py",
    arguments=[
        "--train_size",
        str(0.7),
        "--val_size",
        str(0.2),
        "--test_size",
        str(0.1),
        "--random_state",
        str(10),
        "--cat_feature_cols",
        "sex",
        "--num_feature_cols",
        "length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight",
        "--target_col",
        "rings",
    ],
)

step_preprocess = ProcessingStep(
    name="Abalone-Preprocess", step_args=processor_args, cache_config=step_cache_config
)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:286: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Train

In [13]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

In [14]:
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=train_instance_type.default_value,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_count=train_instance_count,
    output_path=model_path,
    base_job_name="abalone-train",
    instance_type=train_instance_type.default_value,
    role=role,
    sagemaker_session=pipeline_session,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=xgb_num_round,
    max_depth=xgb_max_depth,
    eta=xgb_eta,
    gamma=xgb_gamma,
    min_child_weight=xgb_gamma,
    subsample=xgb_subsample,
)

In [15]:
train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_preprocess.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_train = TrainingStep(
    name="Abalone-Train", step_args=train_args, cache_config=step_cache_config
)

## Evaluation

In [16]:
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

In [17]:
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    volume_size_in_gb=10,
    instance_count=1,
    base_job_name="abalone-postproc",
    role=role,
    sagemaker_session=pipeline_session,
)

In [18]:
eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_preprocess.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/output/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluation.py",
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_eval = ProcessingStep(
    name="Abalone-Evaluate",
    step_args=eval_args,
    cache_config=step_cache_config,
    property_files=[evaluation_report],
)

## Create Model

In [19]:
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

In [20]:
model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

In [21]:
step_create_model = ModelStep(
    name="Abalone-CreateModel",
    step_args=model.create(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),
)

## Register Model

In [22]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

In [23]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

In [24]:
register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=default_model_status,
    model_metrics=model_metrics,
)

step_register = ModelStep(name="Abalone-ModelRegister", step_args=register_args)

## Conditional Check

In [25]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

In [26]:
step_fail = FailStep(
    name="Abalone-FailNotify",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", threshold]),
)

cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=threshold,
)

In [27]:
step_cond = ConditionStep(
    name="Abalone-MSECheck",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model],
    else_steps=[step_fail],
)

# Execute Pipeline

In [28]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline import PipelineExperimentConfig

In [29]:
experiment_name = "AbaloneExperimentsPipeline"

# Pipeline experiment config
abalone_experiments_config = PipelineExperimentConfig(
    experiment_name, ExecutionVariables.PIPELINE_EXECUTION_ID
)

In [30]:
pipeline_name = "AbalonePipeline-SelectExec"

pipeline = Pipeline(
    name=pipeline_name,
    pipeline_experiment_config=abalone_experiments_config,
    parameters=[
        processing_instance_count,
        processing_instance_type,
        train_instance_count,
        train_instance_type,
        xgb_num_round,
        xgb_max_depth,
        xgb_eta,
        xgb_gamma,
        xgb_min_child_weight,
        xgb_subsample,
        model_path,
        default_model_status,
        input_data,
        threshold,
    ],
    steps=[step_preprocess, step_train, step_eval, step_cond],
    sagemaker_session=pipeline_session,
)

In [31]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.c5.xlarge'},
  {'Name': 'TrainingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.c5.xlarge'},
  {'Name': 'XGBNumRounds', 'Type': 'Integer', 'DefaultValue': 50},
  {'Name': 'XGBMaxDepth', 'Type': 'Integer', 'DefaultValue': 5},
  {'Name': 'XGBETA', 'Type': 'Float', 'DefaultValue': 0.2},
  {'Name': 'XGBGamma', 'Type': 'Integer', 'DefaultValue': 4},
  {'Name': 'XGBMinChildWeight', 'Type': 'Integer', 'DefaultValue': 6},
  {'Name': 'XGBSubSample', 'Type': 'Float', 'DefaultValue': 0.7},
  {'Name': 'ModelPath',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-681580474417/Abalone/models/'},
  {'Name': 'ApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApp

In [32]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:681580474417:pipeline/AbalonePipeline-SelectExec',
 'ResponseMetadata': {'RequestId': '18bfd020-370d-4e95-ba59-b674c77832b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '18bfd020-370d-4e95-ba59-b674c77832b8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Tue, 25 Jul 2023 20:12:33 GMT'},
  'RetryAttempts': 0}}

In [33]:
execution = pipeline.start()

In [34]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:681580474417:pipeline/AbalonePipeline-SelectExec',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:681580474417:pipeline/AbalonePipeline-SelectExec/execution/is6ph1fp4go8',
 'PipelineExecutionDisplayName': 'execution-1690315954896',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'AbaloneExperimentsPipeline',
  'TrialName': 'is6ph1fp4go8'},
 'CreationTime': datetime.datetime(2023, 7, 25, 20, 12, 34, 781000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 7, 25, 20, 12, 34, 781000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:681580474417:user-profile/d-biwxq9wovumj/ds-user',
  'UserProfileName': 'ds-user',
  'DomainId': 'd-biwxq9wovumj'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:681580474417:user-profile/d-biwxq9wovumj/ds-user',
  'UserProfileName': 'ds-user',
  'DomainId': 'd-biwxq9wovumj'},
 'ResponseMetadata': {'Req

In [35]:
execution.wait()

# Selective Execution

In [36]:
from sagemaker.workflow.selective_execution_config import SelectiveExecutionConfig

## Setup

In [37]:
pipeline_name = "AbalonePipeline-SelectExec"

pipeline_arn = Pipeline(name=pipeline_name)

# filter for succesfull executions only
pipeline_executions = [
    _exec
    for _exec in pipeline_arn.list_executions()["PipelineExecutionSummaries"]
    if _exec["PipelineExecutionStatus"] == "Succeeded"
]

In [38]:
# get the last successful execution
latest_pipeline_arn = pipeline_executions[0]["PipelineExecutionArn"]

In [39]:
latest_pipeline_arn

'arn:aws:sagemaker:us-east-1:681580474417:pipeline/AbalonePipeline-SelectExec/execution/is6ph1fp4go8'

In [40]:
# list pipeline steps
execution_steps = sm_client.list_pipeline_execution_steps(PipelineExecutionArn=latest_pipeline_arn)[
    "PipelineExecutionSteps"
]

In [41]:
execution_steps[::-1]

[{'StepName': 'Abalone-Preprocess',
  'StartTime': datetime.datetime(2023, 7, 25, 20, 12, 35, 695000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 7, 25, 20, 17, 1, 128000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:681580474417:processing-job/pipelines-is6ph1fp4go8-Abalone-Preprocess-vRZZNaYdbc'}}},
 {'StepName': 'Abalone-Train',
  'StartTime': datetime.datetime(2023, 7, 25, 20, 17, 2, 192000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 7, 25, 20, 19, 33, 989000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:681580474417:training-job/pipelines-is6ph1fp4go8-Abalone-Train-5F3a1Eb2Zd'}}},
 {'StepName': 'Abalone-Evaluate',
  'StartTime': datetime.datetime(2023, 7, 25, 20, 19, 34, 645000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 7, 25, 20, 24, 4, 114000, tzinfo=tzlocal()

In [42]:
# list all configureable pipeline parameters - params can be altered during selective execution
parameters = sm_client.list_pipeline_parameters_for_execution(
    PipelineExecutionArn=latest_pipeline_arn
)["PipelineParameters"]

In [43]:
parameters

[{'Name': 'XGBNumRounds', 'Value': '50'},
 {'Name': 'XGBSubSample', 'Value': '0.7'},
 {'Name': 'XGBGamma', 'Value': '4'},
 {'Name': 'TrainingInstanceCount', 'Value': '1'},
 {'Name': 'XGBMinChildWeight', 'Value': '6'},
 {'Name': 'XGBETA', 'Value': '0.2'},
 {'Name': 'ApprovalStatus', 'Value': 'PendingManualApproval'},
 {'Name': 'ProcessingInstanceCount', 'Value': '1'},
 {'Name': 'ProcessingInstanceType', 'Value': 'ml.c5.xlarge'},
 {'Name': 'MseThreshold', 'Value': '6.0'},
 {'Name': 'ModelPath',
  'Value': 's3://sagemaker-us-east-1-681580474417/Abalone/models/'},
 {'Name': 'XGBMaxDepth', 'Value': '5'},
 {'Name': 'TrainingInstanceType', 'Value': 'ml.c5.xlarge'},
 {'Name': 'InputData',
  'Value': 's3://sagemaker-us-east-1-681580474417/sample-dataset/abalone/abalone.csv'}]

## Selective Execution Scenarios

### Execute Single Pipeline Step with Changes to Parameters

I would like to retrain the model in hopes to get a better performant model with different hyper parameters without retraining the whole pipeline flow, this is useful in scenarios where a DS would prefer to just test custom hyper parameters before executing the full pipeline flow

In [44]:
selective_execution_config = SelectiveExecutionConfig(
    source_pipeline_execution_arn=pipeline_executions[0]["PipelineExecutionArn"],
    selected_steps=["Abalone-Train"],
)

In [45]:
select_execution = pipeline_arn.start(
    selective_execution_config=selective_execution_config,
    parameters={
        "XGBNumRounds": 120,
        "XGBSubSample": 0.9,
        "XGBGamma": 2,
        "XGBMinChildWeight": 4,
        "XGBETA": 0.25,
        "XGBMaxDepth": 12,
    },
)

Before:

![Before Single Step](./media/BeforeSingleStep.jpg)

After

![After Single Step](./media/AfterSingleStep.jpg)

### Execute Multiple Contiguous Pipeline Steps or the Entire Pipeline Manually

In [46]:
selective_execution_config = SelectiveExecutionConfig(
    source_pipeline_execution_arn=latest_pipeline_arn,
    selected_steps=["Abalone-Train", "Abalone-Evaluate", "Abalone-MSECheck"],
)

In [ ]:
select_execution = pipeline_arn.start(
    selective_execution_config=selective_execution_config,
    parameters={
        "XGBNumRounds": 120,
        "XGBSubSample": 0.9,
        "XGBGamma": 2,
        "XGBMinChildWeight": 4,
        "XGBETA": 0.25,
        "XGBMaxDepth": 12,
        "MseThreshold": 6,
    },
)

Execute Contiguous Blocks of the Pipeline


![Multiple Steps](./media/MultipleSteps.jpg)


OR Execute the full pipeline 

![Full Steps](./media/FullPipeline.jpg)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-pipelines|selective-step-execution|Selective_Execution_Abalone.ipynb)
